In [ ]:
#Lag en modell som predikerer tilsvarende XOR-Operatoren.
#Før du optimaliserer denne modellen må du initialisere modellvariablene med tilfeldige tall, for eksempel mellom -1 og 1.
#Visualiser både når modellen konvergerer og ikke konvergerer mot en riktig modell.

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt